In [ ]:
# Carl Tape, Lion Krischer, Amanda McPherson
# GEOS626, Applied Seismology, University of Alaska Fairbanks

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from obspy.clients.fdsn import Client
from obspy.core import UTCDateTime

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 5

In [ ]:
# getting the inventory

client = Client("IRIS")

starttime = UTCDateTime(2004, 12, 26, 0, 58, 50)
endtime   = starttime + ( 5 * 24 * 60 * 60)                                # 5 days after the start time

inv = client.get_stations(network="G", station="CAN", channel="LHZ", starttime=starttime, endtime=endtime, level="response")

# check the page - https://docs.obspy.org/packages/obspy.core.inventory.html, 
# for detailed information on the inventory object

print(inv)
print(inv[0])
print(inv[0][0])
print(inv[0][0][0])

samples_per_second = inv[0][0][0].sample_rate
Nyquist_frequency  = ( 1 / samples_per_second ) / 2

print('\n')
print(f'Nyquist frequency = {Nyquist_frequency} Hz \n')

In [ ]:
# inspect the instrument response

# check the page - https://docs.obspy.org/packages/autogen/obspy.core.inventory.inventory.Inventory.plot_response.html,
# for detailed information on plotting response using this method

inv.plot_response(min_freq=0.0001);

In [ ]:
# see more details about the response

print(inv[0][0][0].response)

In [ ]:
# leave out the digitizer and FIR filters, and examine the "simple" pole-zero plot.
# compare it with the plot above.

# TRY OUT: Replace VEL with ACC or DISP to see response for velocity or acceleration

# check this page - https://docs.obspy.org/packages/autogen/obspy.core.inventory.channel.Channel.plot.html
# for detailed information on plotting response using this method

inv[0][0][0].plot(min_freq=0.0001, start_stage=1, end_stage=1, output="VEL");

In [ ]:
# plotting instrument responses together for comparison

f = np.logspace(-4.0, 2.0, num=100)                  # set of frequencies to evaluate the response

Id = inv[0][0][0].response.get_evalresp_response_for_frequencies(f, output='DISP')
Iv = inv[0][0][0].response.get_evalresp_response_for_frequencies(f, output='VEL')
Ia = inv[0][0][0].response.get_evalresp_response_for_frequencies(f, output='ACC')

max_Id   = max(Id)
f_Id_max = f[np.nonzero(Id == max_Id)]
max_Iv   = max(Iv)
f_Iv_max = f[np.nonzero(Iv == max_Iv)]
max_Ia   = max(Ia)
f_Ia_max = f[np.nonzero(Ia == max_Ia)]

fig, ax = plt.subplots(3, 2, figsize=(14,16))

ax[0,0].semilogx(f, np.angle(Id)*180/np.pi, color='b')
ax[0,0].set(xlabel='Frequency (Hz)', ylabel='Phase (deg)')
ax[0,0].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])

ax[0,1].loglog(f, abs(Id), color='b')
ax[0,1].set(xlabel='Frequency (Hz)', ylabel='Amplitude')
ax[0,1].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])
ax[0,1].set_title('(m to counts) max = %.2e at %.2e Hz'% (np.real(max_Id), f_Id_max), fontsize=11)

ax[1,0].semilogx(f, np.angle(Iv)*180/np.pi, color='b')
ax[1,0].set(xlabel='Frequency (Hz)', ylabel='Phase (deg)')
ax[1,0].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])

ax[1,1].loglog(f, abs(Iv), color='b')
ax[1,1].set(xlabel='Frequency (Hz)', ylabel='Amplitude')
ax[1,1].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])
ax[1,1].set_title('(m/s to counts) max = %.2e at %.2e Hz'% (np.real(max_Iv), f_Iv_max), fontsize=11)

ax[2,0].semilogx(f, np.angle(Ia)*180/np.pi, color='b')
ax[2,0].set(xlabel='Frequency (Hz)', ylabel='Phase (deg)')
ax[2,0].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])

ax[2,1].loglog(f, abs(Ia), color='b')
ax[2,1].set(xlabel='Frequency (Hz)', ylabel='Amplitude')
ax[2,1].set_title('(m/s2 to counts) max = %.2e at %.2e Hz'% (np.real(max_Ia), f_Ia_max), fontsize=11)
ax[2,1].axvline(Nyquist_frequency, c='r', lw=0.5, ls='--', dashes=[12,12])

plt.subplots_adjust(hspace=0.25)